In [43]:
import os 
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import warnings
warnings.simplefilter('ignore')

In [44]:
print(os.getcwd())


/Users/ellenwu/homework2-2/submission3/data-code


In [ ]:
print(os.path.exists('/Users/ellenwu/homework2-2/data/*/input/HCRISv1996'))
print(os.path.exists('/Users/ellenwu/homework2-2/data/*/input/HCRISv2010/HCRISv2010'))

input("input to continue")
# Call individual scripts -------------------------------------------------
exec(open('/Users/ellenwu/homework2-2/submission3/data-code/H1_HCRISv1996.py').read())
exec(open('/Users/ellenwu/homework2-2/submission3/data-code/H1_HCRISv2010.py').read())

True
True


In [45]:
# Read and combine data ---------------------------------------------------
HCRIS_v1996 = pd.read_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_v1996.csv')
HCRIS_v2010 = pd.read_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_v2010.csv')

In [46]:
# Create missing variables for columns introduced in v2010
HCRIS_v1996['hvbp_payment'] = np.nan
HCRIS_v1996['hrrp_payment'] = np.nan

In [47]:
# Combine v1996 and v2010 datasets
HCRIS_Data = pd.concat([HCRIS_v1996, HCRIS_v2010])

In [48]:
# Convert date columns to datetime format
for col in ['fy_end', 'fy_start', 'date_processed', 'date_created']:
    HCRIS_Data[col] = pd.to_datetime(HCRIS_Data[col], format='%m/%d/%Y')

In [49]:
# Convert to absolute values
HCRIS_Data['tot_discounts'] = HCRIS_Data['tot_discounts'].abs()
HCRIS_Data['hrrp_payment'] = HCRIS_Data['hrrp_payment'].abs()

In [50]:
# Extract fiscal year and sort
HCRIS_Data['fyear'] = HCRIS_Data['fy_end'].dt.year
HCRIS_Data = HCRIS_Data.sort_values(by=['provider_number', 'fyear']).drop(columns=['year'], errors='ignore')

In [51]:
# Count hospitals per year
hospital_counts = HCRIS_Data.groupby('fyear').size()

In [53]:
# Clean data --------------------------------------------------------------

# Create count of reports by hospital fiscal year
HCRIS_Data['total_reports'] = HCRIS_Data.groupby(['provider_number', 'fyear'])['provider_number'].transform('count')
HCRIS_Data['report_number'] = HCRIS_Data.groupby(['provider_number', 'fyear']).cumcount() + 1

In [54]:

# Identify hospitals with one report per year
unique_hcris1 = HCRIS_Data[HCRIS_Data['total_reports'] == 1].drop(columns=['report', 'total_reports', 'report_number', 'npi', 'status'], errors='ignore')
unique_hcris1['source'] = 'unique reports'
print(unique_hcris1)

      provider_number   fy_start     fy_end date_processed date_created  \
2440            10001 2016-10-01 2017-09-30     2018-07-19   2018-07-10   
347             10005 2016-10-01 2017-09-30     2018-03-19   2018-03-12   
4767            10006 2017-07-01 2018-06-30     2018-12-20   2018-12-13   
136             10007 2016-10-01 2017-09-30     2018-03-07   2018-03-05   
979             10008 2017-01-01 2017-12-31     2018-06-04   2018-05-26   
...               ...        ...        ...            ...          ...   
1566           673060 2017-01-01 2017-12-31     2018-06-22   2018-06-20   
3405           673061 2017-06-01 2018-05-31     2018-11-02   2018-11-01   
2587           673062 2017-01-01 2017-12-31     2018-07-19   2018-06-26   
5065           673063 2017-08-01 2018-07-31     2019-01-24   2019-01-03   
3052           673064 2017-04-01 2018-03-31     2018-09-17   2018-09-10   

       beds   tot_charges  tot_discounts  tot_operating_exp   ip_charges  ...  \
2440  387.0  1.707

In [33]:
# Identify hospitals with multiple reports per year
duplicate_hcris = HCRIS_Data[HCRIS_Data['total_reports'] > 1].copy()
print(duplicate_hcris)

      report  provider_number  npi   fy_start     fy_end date_processed  \
15    603594            50313  NaN 2017-01-01 2017-04-30     2017-10-17   
1589  628008            50313  NaN 2017-05-01 2017-12-31     2018-06-25   
89    619590            50528  NaN 2017-01-01 2017-04-30     2018-02-28   
1598  628017            50528  NaN 2017-05-01 2017-12-31     2018-06-25   
14    603229            50557  NaN 2017-01-01 2017-04-30     2017-10-17   
...      ...              ...  ...        ...        ...            ...   
1884  628432           514001  NaN 2017-09-27 2017-12-31     2018-06-26   
52    614315           670058  NaN 2017-01-01 2017-06-30     2018-01-19   
2706  630024           670058  NaN 2017-07-01 2017-12-31     2018-07-24   
53    614316           670095  NaN 2017-01-01 2017-06-30     2018-01-19   
2712  630030           670095  NaN 2017-07-01 2017-12-31     2018-07-24   

     date_created  status   beds  tot_charges  ...  \
15     2017-10-11       1   81.0  150375828.0

In [55]:
# Calculate elapsed time between fy start and fy end for hospitals with multiple reports
duplicate_hcris['time_diff'] = (duplicate_hcris['fy_end'] - duplicate_hcris['fy_start']).dt.days
duplicate_hcris['total_days'] = duplicate_hcris.groupby(['provider_number', 'fyear'])['time_diff'].transform('sum')

In [56]:
# Hospitals where total days < 370, sum values
unique_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] < 370].groupby(['provider_number', 'fyear']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris2['source'] = 'total for year'

In [57]:
# Hospitals with reports exceeding 370 days
duplicate_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] >= 370].copy()
duplicate_hcris2['max_days'] = duplicate_hcris2.groupby(['provider_number', 'fyear'])['time_diff'].transform('max')
duplicate_hcris2['max_date'] = duplicate_hcris2.groupby(['provider_number', 'fyear'])['fy_end'].transform('max')

In [58]:
# Primary report selection
unique_hcris3 = duplicate_hcris2[(duplicate_hcris2['max_days'] == duplicate_hcris2['time_diff']) &
                                  (duplicate_hcris2['time_diff'] > 360) &
                                  (duplicate_hcris2['max_date'] == duplicate_hcris2['fy_end'])]
unique_hcris3 = unique_hcris3.drop(columns=['report', 'total_reports', 'report_number', 'npi', 'status', 'max_days', 'time_diff', 'total_days', 'max_date'], errors='ignore')
unique_hcris3['source'] = 'primary report'

In [59]:
# Remaining hospitals with reports covering more than one full year
duplicate_hcris3 = duplicate_hcris2[~duplicate_hcris2.index.isin(unique_hcris3.index)].copy()
duplicate_hcris3['time_diff'] = duplicate_hcris3['time_diff'].astype(int)
duplicate_hcris3['total_days'] = duplicate_hcris3['total_days'].astype(int)

cols_to_weight = ['tot_charges', 'tot_discounts', 'tot_operating_exp', 'ip_charges',
                  'icu_charges', 'ancillary_charges', 'tot_discharges', 'mcare_discharges',
                  'mcaid_discharges', 'tot_mcare_payment', 'secondary_mcare_payment',
                  'hvbp_payment', 'hrrp_payment']

duplicate_hcris3[cols_to_weight] = duplicate_hcris3[cols_to_weight].mul(
    duplicate_hcris3['time_diff'] / duplicate_hcris3['total_days'], axis=0)

In [60]:
# Weighted average
unique_hcris4 = duplicate_hcris3.groupby(['provider_number', 'fyear']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris4['source'] = 'weighted_average'

In [61]:
# Combine final datasets
HCRIS_Data = pd.concat([unique_hcris1, unique_hcris2, unique_hcris3, unique_hcris4])
HCRIS_Data = HCRIS_Data.rename(columns={'fyear': 'year'}).sort_values(by=['provider_number', 'year'])

In [62]:
# Save final data ---------------------------------------------------------
HCRIS_Data.to_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_Data.csv', index=False)